In [ ]:
# --- SETUP ---
from google.cloud import bigquery

PROJECT_ID = "qwiklabs-gcp-04-0dc8cd335953"   # Replace with your project ID
DATASET_ID = "aurora_bay"
TABLE_ID = "faqs_raw"

bq_client = bigquery.Client(project=PROJECT_ID)

# --- CREATE DATASET IF NOT EXISTS ---
dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset_ref.location = "US"
bq_client.create_dataset(dataset_ref, exists_ok=True)

print(f"Dataset {DATASET_ID} is ready.")

# --- LOAD CSV FROM GCS INTO BIGQUERY ---
gcs_uri = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,   # Skip header row
    autodetect=True,       # Let BigQuery detect schema
    write_disposition="WRITE_TRUNCATE"  # Overwrite table each run
)

load_job = bq_client.load_table_from_uri(
    gcs_uri,
    f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}",
    job_config=job_config
)

load_job.result()  # Wait for job to finish

# --- CONFIRM LOAD ---
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}")
print(f"Loaded {table.num_rows} rows and {len(table.schema)} columns into {table.full_table_id}")

Dataset aurora_bay is ready.
Loaded 50 rows and 2 columns into qwiklabs-gcp-04-0dc8cd335953:aurora_bay.faqs_raw


In [ ]:
from google.cloud import bigquery

PROJECT_ID = "qwiklabs-gcp-04-0dc8cd335953"   # replace
DATASET_ID = "aurora_bay"
OUTPUT_TABLE = f"{PROJECT_ID}.{DATASET_ID}.faqs_with_embeddings"

bq_client = bigquery.Client(project=PROJECT_ID)

# Drop table if exists
bq_client.query(f"DROP TABLE IF EXISTS `{OUTPUT_TABLE}`").result()

# Create new table
bq_client.query(f"""
CREATE TABLE `{OUTPUT_TABLE}` (
  id INT64,
  question STRING,
  answer STRING,
  embedding ARRAY<FLOAT64>
)
""").result()

print("Created table:", OUTPUT_TABLE)


Created table: qwiklabs-gcp-04-0dc8cd335953.aurora_bay.faqs_with_embeddings


In [ ]:
import bigframes.pandas as bpd

RAW_TABLE = f"{PROJECT_ID}.{DATASET_ID}.faqs_raw"

df_faqs = bpd.read_gbq(RAW_TABLE)
df_faqs.peek()

,string_field_0,string_field_1
36,Where is the nearest university or college?,There’s no university in Aurora Bay itself. Th...
42,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...
25,How do I contact the Aurora Bay Fire Department?,The volunteer-based Aurora Bay Fire Department...
4,Are there guided tours for Northern Lights vie...,Yes. Several local outfitters offer guided nig...
14,What is the population of Aurora Bay?,Aurora Bay has a population of approximately 3...


In [ ]:
from vertexai.language_models import TextEmbeddingModel

embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")
print("Embedding model loaded.")

Embedding model loaded.


/usr/local/lib/python3.12/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [ ]:
rows_to_insert = []

for idx, row in df_faqs.iterrows():
    question = str(row["string_field_0"])
    answer = str(row["string_field_1"])

    # Combine Q + A into one text block
    text = question + "\n" + answer

    # Generate embedding
    emb = embedding_model.get_embeddings([text])[0].values
    emb = [float(x) for x in emb]  # convert to Python floats

    rows_to_insert.append({
        "id": int(idx),
        "question": question,
        "answer": answer,
        "embedding": emb
    })

# Insert into BigQuery
errors = bq_client.insert_rows_json(OUTPUT_TABLE, rows_to_insert)

if errors:
    print("Errors inserting rows:", errors)
else:
    print("Embeddings successfully inserted into BigQuery.")

Embeddings successfully inserted into BigQuery.


In [ ]:
table = bq_client.get_table(OUTPUT_TABLE)
print(f"Table {OUTPUT_TABLE} now contains {table.num_rows} rows.")

Table qwiklabs-gcp-04-0dc8cd335953.aurora_bay.faqs_with_embeddings now contains 0 rows.


In [ ]:
from google.cloud import bigquery
import bigframes.pandas as bpd
from bigframes.ml.llm import TextEmbeddingGenerator
from typing import List

In [ ]:
PROJECT_ID = "qwiklabs-gcp-04-0dc8cd335953"   # Replace with your project ID
DATASET_ID = "aurora_bay"

EMBEDDINGS_TABLE = f"{PROJECT_ID}.{DATASET_ID}.faqs_with_embeddings"

bq_client = bigquery.Client(project=PROJECT_ID)


In [ ]:
QUERY_EMBEDDING_MODEL = "text-embedding-005"
embedding_model = TextEmbeddingGenerator(model_name=QUERY_EMBEDDING_MODEL)

In [ ]:
def get_similar_faqs(query: str, top_k: int = 3) -> List[dict]:
    """
    Performs a vector search using ML.DISTANCE() on ARRAY<FLOAT64>.
    Returns the top_k most similar FAQs.
    """

    # Step 1 — Generate embedding for the query
    df_query = bpd.DataFrame({"content": [query]})
    query_emb_bf = embedding_model.predict(df_query)
    query_emb_list = query_emb_bf['ml_generate_embedding_result'].to_pandas().iloc[0]

    # Convert embedding to SQL array literal
    query_emb_sql = str(list(query_emb_list))

    # Step 2 — Build vector search SQL
    sql = f"""
    SELECT
        question,
        answer,
        ML.DISTANCE(
            embedding,
            ARRAY<FLOAT64>{query_emb_sql},
            'COSINE'
        ) AS similarity_score
    FROM `{EMBEDDINGS_TABLE}`
    ORDER BY similarity_score
    LIMIT {top_k}
    """

    # Step 3 — Execute query
    results = bq_client.query(sql).result()

    # Step 4 — Format results
    faqs = []
    for row in results:
        faqs.append({
            "question": row.question,
            "answer": row.answer,
            "similarity_score": float(row.similarity_score)
        })

    return faqs

In [ ]:
def chatbot():
    print("Welcome to the Aurora Bay FAQ Chatbot!")
    print("Type 'exit' or 'quit' to stop.")

    while True:
        user_query = input("\nYour question: ")

        if user_query.lower() in ("exit", "quit"):
            print("Goodbye!")
            break

        print(f"\nSearching for answers to: {user_query}")

        results = get_similar_faqs(user_query, top_k=3)

        if not results:
            print("No matching FAQs found. Try rephrasing.")
            continue

        print("\nTop matches:")
        for i, faq in enumerate(results, start=1):
            print(f"\n--- Result {i} (Similarity: {faq['similarity_score']:.4f}) ---")
            print(f"Q: {faq['question']}")
            print(f"A: {faq['answer']}")

In [ ]:
chatbot()

Welcome to the Aurora Bay FAQ Chatbot!
Type 'exit' or 'quit' to stop.

Your question: Where is the nearest university or college?

Searching for answers to: Where is the nearest university or college?


/usr/local/lib/python3.12/dist-packages/bigframes/dtypes.py:987: JSONDtypeWarning: JSON columns will be represented as pandas.ArrowDtype(pyarrow.json_())
instead of using `db_dtypes` in the future when available in pandas
(https://github.com/pandas-dev/pandas/issues/60958) and pyarrow.
  warnings.warn(msg, bigframes.exceptions.JSONDtypeWarning)
/usr/local/lib/python3.12/dist-packages/bigframes/dtypes.py:987: JSONDtypeWarning: JSON columns will be represented as pandas.ArrowDtype(pyarrow.json_())
instead of using `db_dtypes` in the future when available in pandas
(https://github.com/pandas-dev/pandas/issues/60958) and pyarrow.
  warnings.warn(msg, bigframes.exceptions.JSONDtypeWarning)



Top matches:

--- Result 1 (Similarity: 0.3213) ---
Q: Where is the nearest university or college?
A: There’s no university in Aurora Bay itself. The nearest college campus is located in Kodiak, roughly a 2-hour ferry trip away, depending on weather.

--- Result 2 (Similarity: 0.4548) ---
Q: What educational facilities are available for children?
A: Aurora Bay has one elementary school, one middle school, and a combined high school. They’re managed by the Aurora Bay School District.

--- Result 3 (Similarity: 0.5590) ---
Q: Does Aurora Bay have a public library?
A: Yes. The Aurora Bay Public Library is located on Main Street, next to the town’s post office.
